# **Function Calling to Generate JSON data & Validate using Pydantic**
#### **✅ Problem:**
How can we get structured, validated JSON output reliably from a language model?
####**🛠️ Solution:**
* This lab demonstrates the use of OpenAI Function Calling as a contract-driven way of interacting with LLMs.
* Using Pydantic Validate the generated data is matching the type or not?

### **Install Dependencies**

In [2]:
!pip install openai pydantic


###**Retrive API key from Secrets and Set as an ENV**

In [3]:
# Retrieve the API key from Colab's secrets
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Set OPENAI_API_KEY as an ENV
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [4]:
from openai import OpenAI
client = OpenAI()

###**Python Code**

In [11]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": "Extract name, age, and city from the sentence: Ritu Patil is 30 and lives in Chicago. she earns $6000 per month",
        }
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "extract_person_info",
                "description": "Extract name, age, and city from text",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"},
                        "age": {"type": "integer"},
                        "salary": {"type": "integer"},
                        "city": {"type": "string"},
                    },
                    "required": ["name", "age","salary","city"],
                },
            },
        }
    ],
    tool_choice="auto",
)

# Extract the structured response
tool_calls = response.choices[0].message.tool_calls
arguments = tool_calls[0].function.arguments

import json
parsed = json.loads(arguments)
print(parsed)

{'name': 'Ritu Patil', 'age': 30, 'salary': 6000, 'city': 'Chicago'}


In [12]:
print("Yearly Bonus = $", parsed['salary']*2)

Yearly Bonus = $ 12000


#**Validate using Pydantic**

In [16]:
from pydantic import BaseModel, ValidationError
import json

# Step 1: Define a Pydantic model
class PersonInfo(BaseModel):
    name: str
    age: int
    salary: int
    city: str

# Step 2: Get the structured output from OpenAI function call
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": "Extract name, age, and city from the sentence: Ritu Patil is 30 and lives in Chicago. she earns $6000 per month",
        }
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "extract_person_info",
                "description": "Extract name, age, and city and salary from text",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"},
                        "age": {"type": "integer"},
                       # "salary": {"type": "integer"},
                        "salary": {"type": "string"},
                        "city": {"type": "string"},
                    },
                    "required": ["name", "age", "salary" "city"],
                },
            },
        }
    ],
    tool_choice="auto",
)

# Step 3: Extract and parse tool call arguments
tool_calls = response.choices[0].message.tool_calls
arguments = tool_calls[0].function.arguments
parsed = json.loads(arguments)
print(parsed)

# Step 4: Validate using Pydantic
try:
    validated = PersonInfo(**parsed)
    print("\n✅ Validated:", validated)
except ValidationError as ve:
    print("❌ Validation Error:", ve)


{'name': 'Ritu Patil', 'age': 30, 'salary': '$6000 per month', 'city': 'Chicago'}
❌ Validation Error: 1 validation error for PersonInfo
salary
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='$6000 per month', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing


In [14]:
print("Yearly Bonus =", parsed['salary']*2)

Yearly Bonus = $6000 per month$6000 per month


In [15]:
print(parsed['age']*2)

60
